# UI with drop down menus using Gradio Day to print streaming Articles!

Or my personal chatgpt?!

In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [95]:
import gradio as gr # oh yeah!

In [96]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key not set


In [97]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [93]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [90]:
system_prompt = "You are provided with a list of links found on a wikipedia search result page. \
You are able to decide which of the links would be most relevant to include in a creative snapshot summary about the topic searched on wikipedia., \
such as links technical terms, important people listed in the wikipedia description, links to any historical references, books, movies, awards, etc,.\n"
system_prompt += "For a company cover their founders, key persons, competing founders, products, company culture, revenue and future product outlook."

system_prompt += "For a famous persona cover what they are famous for, their work, especially the popular ones. Why did this work become popular. Awards they have won. \
They education, info about their spouses, partners, family and kids. What direction they are thinking of working in in the future (or have they stopped working)? \
Struggles they have faced and what it took them to get where they are."
system_prompt += "You should respond in Markdown."
system_prompt += "The examples are over! If I have more details I will share it after this sentence."

In [91]:
def stream_gpt_with_tone(user_prompt,system_prompt_param):
    messages = [
        {"role": "system", "content": system_prompt_param},
        {"role": "user", "content": user_prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

def stream_claude_with_tone(user_prompt,system_prompt_param):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_prompt_param,
        messages=[
            {"role": "user", "content": user_prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [92]:
def stream_magazine_article(company_name, url, model, tone):
    yield ""
    system_prompt_param = system_prompt
    if len(tone)>0:
        system_prompt_param+=f" Btw, you have a {tone} tone when creating the brochure, so don't be afraid to really lean into this tone!"
    user_prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    user_prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt_with_tone(user_prompt, system_prompt_param)
    elif model=="Claude":
        result = stream_claude_with_tone(user_prompt, system_prompt_param)
    else:
        raise ValueError("Unknown model")
    yield from result

In [94]:
view = gr.Interface(
    fn=stream_magazine_article,
    inputs=[
        gr.Textbox(label="Company or Person name:"),
        gr.Textbox(label="Wikipedia url"),
        gr.Dropdown(["GPT", "Claude"], label="Select model"),
        gr.Dropdown(["", "Funny", "Snarky","Very Intellectual", "Poetic"], label="Tone")
    ],
    outputs=[gr.Markdown(label="Newsletter Article:")],
    flagging_mode="never"
)
view.launch()
#https://en.wikipedia.org/wiki/Meryl_Streep

* Running on local URL:  http://127.0.0.1:7890
* To create a public link, set `share=True` in `launch()`.


# END